In [10]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리
from scipy.stats import norm #p-value 를 위함


mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [2]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [95]:
def detection(final_sequence, logit_shape, gamma) :
    
    input_ids = tokenizer.encode(final_sequence, return_tensors='tf')
    input_ids_list = input_ids.numpy().tolist()[0]
    prev_tk = input_ids[0]
    green_counts = 0
    
    for id in input_ids_list :
        seed = id % logit_shape
        #print(f"seed : {seed[-1]}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
        
        if id in green_list :
            green_counts = green_counts + 1
        prev_id = id

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list)
    red_proportion = 1-green_proportion
    print(f"green list 단어 비율 : {green_proportion}")
    print(f"red list 단어 비율 : {red_proportion}")
    expected_proportion = 0.5 #green list 를 모를 것이다 -> H0
    std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    z_score = (green_proportion - expected_proportion) / std_error
    p_value = 2 * norm.sf(np.abs(z_score))
    
    print(f"Z-score: {z_score}, P-value: {p_value}")
    if p_value < 0.05 :
        return True
    else :
        return False

#g=0.7, d=10.0 ->red list 단어 비율 : 0.580000000000000
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에는 책 읽는 즐거움이란 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로"
#g=0.5, d=50.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=20.0 -> red list 단어 비율 : 0.53
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 "
#g=0.5, d=10.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=5.0 -> red list 단어 비율 : 0.505050505050505
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책과 나눔'이라는 주제 하에 지난달 27일 개막한 '책과 나눔'전을 통해 처음 선보였으며, 오는 9월까지 계속된다. 특히, 전시 기간 동안 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해"
#g=0.5, d=1.0 -> red list 단어 비율 : 0.6
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다.이번 전시는 ‘책 읽는 즐거움’을 주제로 한 ‘책 읽는 즐거움’ 전시를 비롯해 ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움"
#g=1.0, d=0.0 -> red list 단어 비율 : 0.5700000000000001
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에서는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는"

#g=0.5
#sentence = "아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다. 오늘은 전국이 대체로 맑겠습니다. 하지만 제주도는 구름이 많이 끼겠고요. 강원 영동 지역은 밤 한때 비가 조금 내리겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다. 서울이 4도, 대구 6도로 출발하겠습니다. 낮 기온은 서울이 13도, 대구와 광주 16도로 오늘과 비슷하겠습니다. 바다의 물결은 동해 먼바다에서 최고 2.5미터까지 높게 일겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다."

#사람
#sentence = "안정된 순방향 전파: 만약 우리가 오프셋 δ를 0으로 초기화하고 스케일 γ를 1로 초기화한다면, 각 출력 활성화는 단위 분산을 갖게 됩니다. 일반적인 네트워크에서는 이는 초기화 과정에서 순방향 전파 중에 분산이 안정적으로 유지됨을 보장합니다. 하지만 잔차 네트워크에서는 분산은 여전히 새로운 변동 요소가 각 레이어에서 입력에 추가되면서 증가해야 합니다. 그러나 잔차 블록마다 분산은 선형적으로 증가할 것입니다. k번째 레이어는 k의 기존"

#사람
#sentence = "이건 사람이 작성한 글입니다. 절대 절대 p-value 가 낮게 나와서도 안되고 z-score가 0 근처여서도 안됩니다. 제발 좀 잘 나와보자 제발제발 할말이 없네 이게 다야"
detection(sentence, 25600, 0.5)

green list 단어 개수 : 42
red list 단어 개수 : 58
green list 단어 비율 : 0.42
red list 단어 비율 : 0.5800000000000001
Z-score: -0.16000000000000003, P-value: 0.8728810742171342


False

: 

In [65]:

with open('perplexity_g=0.5_d=1.0.txt', 'w') as f:  #file내용 clear
    f.write('')

for line in lines:
    line = line.strip()
    print(f"prompt : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
            
    with mirrored_strategy.scope():
        for _ in tqdm(range(100)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 100:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 0.7
            delta = 10.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(f"updated Logits : {Logits}")
            
            # softmax 계산을 위한 분모
            denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list

            # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            softmax_tensor = Logits.numpy()
            softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 데이터 타입 확인 및 조정
            add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"updated Logits Max value : {add_token_id}")
            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            nlls.append(np.log(selected_probability))

            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)
        print(f"is this sentence written by machine? {detection(final_token, Logits.shape[-1], gamma)}")

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        perplexity_str = str(perplexity.numpy())  # perplexity를 문자열로 변환
        with open('perplexity_g=0.5_d=1.0.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(perplexity_str + '\n')

prompt : 아침 일과 시작하기


  0%|          | 0/100 [00:00<?, ?it/s]

seed : 9154
green list : [18100 37771 17147 ... 40004 30326 39041] (35840,)


  1%|          | 1/100 [00:00<00:20,  4.73it/s]

next token : 아침 일과 시작하기 전에
seed : 10993
green list : [45309 34829 24174 ... 46073 12161 17041] (35840,)


...

  6%|▌         | 6/100 [00:01<00:16,  5.55it/s]

next token : 가족과 함께 하는 저녁 식사 시간도 마련했다.
이날
seed : 7060
green list : [40113  5482  8283 ... 30040 15927 29322] (35840,)
next token : 가족과 함께 하는 저녁 식사 시간도 마련했다.
이날 행사


  7%|▋         | 7/100 [00:01<00:16,  5.52it/s]

seed : 11570
green list : [37652 33290 31512 ...  3380  7989 32813] (35840,)
next token : 가족과 함께 하는 저녁 식사 시간도 마련했다.
이날 행사에는


KeyboardInterrupt: 

In [35]:
np.random.seed(9154)
green_list_size = int(Logits.shape[-1] * gamma)  #25600
green_list = np.random.choice(indices, green_list_size, replace=False)
print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

green list : [18100 37771 17147 ... 26412 26911 43793] (25600,)
